In [2]:
import requests
import datetime
import pprint
import sqlite3
import json
import pandas as pd

In [3]:
last_week = (datetime.date.today() - datetime.timedelta(days=7)).strftime("%Y-%m-%d")
link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={last_week}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
conn = sqlite3.connect('NASA_NEO.db')

Exploration of ETL

In [4]:
# for date, neo_list in neo_data.items():
#     for neo in neo_list:
#         cursor.execute('''
#                     INSERT INTO neo VALUES (?, ?, ?, ?, ?, ?)
#                     ''', (
#                         neo['id'],
#                         neo['name'],
#                         neo['is_potentially_hazardous_asteroid'],
#                         neo['is_sentry_object'],
#                         neo['close_approach_data'],
#                         neo['estimated_diameter']
#                     ))
# conn.commit()

ELT

In [5]:
def load_data():
    response = requests.get(link)
    data = response.json()
    neo_data = data['near_earth_objects']
  
    #pprint.pprint(neo_data)

    cursor = conn.cursor()

    cursor.executescript('''

        DROP TABLE IF EXISTS neo;

        CREATE TABLE neo (
                        id text,
                        data json
        )
    ''')

    for date, neo_list in neo_data.items():
        for neo in neo_list:
            cursor.execute('INSERT INTO neo VALUES (?, ?)',
                        (neo['id'], json.dumps(neo)))
    conn.commit()

Exploration of SQL transformation

In [6]:
# df = pd.read_sql_query('''
#                        select json_extract(data, "$.name") as name,
#                        json_extract(data, "$.id") as ID,
#                        json_extract(data, "$.is_potentially_hazardous_asteroid") as potentially_hazardous
#                        from neo;
#                        ''', conn)

# df["potentially_hazardous"] = df["potentially_hazardous"].astype(bool)

Pandas transformation

In [7]:
df = pd.read_sql_query('''SELECT data FROM neo''', conn)

df['data'] = df['data'].apply(json.loads)

json_df = pd.json_normalize(df['data'])

json_df

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max
0,2137084,2137084,137084 (1998 XS16),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2137084,16.49,False,"[{'close_approach_date': '2023-10-01', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/2137084?ap...,1.338305,2.992540,1338.304618,2992.540101,0.831584,1.859478,4390.763323,9818.045264
1,2252399,2252399,252399 (2001 TX44),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2252399,19.45,True,"[{'close_approach_date': '2023-10-01', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/2252399?ap...,0.342417,0.765667,342.416731,765.667087,0.212768,0.475763,1123.414507,2512.031205
2,3342645,3342645,(2006 SG7),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3342645,22.82,False,"[{'close_approach_date': '2023-10-01', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3342645?ap...,0.072536,0.162196,72.536229,162.195940,0.045072,0.100784,237.979763,532.138928
3,3451020,3451020,(2009 EG3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3451020,21.31,False,"[{'close_approach_date': '2023-10-01', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3451020?ap...,0.145397,0.325117,145.396843,325.117225,0.090345,0.202018,477.023778,1066.657595
4,3606252,3606252,(2012 OO),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3606252,20.65,True,"[{'close_approach_date': '2023-10-01', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3606252?ap...,0.197040,0.440595,197.040262,440.595420,0.122435,0.273773,646.457574,1445.523079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6647,54305672,54305672,(2022 SK5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54305672,23.69,False,"[{'close_approach_date': '2022-09-24', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54305672?a...,0.048591,0.108653,48.590904,108.652564,0.030193,0.067514,159.418981,356.471678
6648,54308110,54308110,(2022 SF19),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54308110,29.31,False,"[{'close_approach_date': '2022-09-24', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54308110?a...,0.003652,0.008167,3.652204,8.166575,0.002269,0.005074,11.982296,26.793227
6649,54311855,54311855,(2022 SL48),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54311855,26.73,False,"[{'close_approach_date': '2022-09-24', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54311855?a...,0.011983,0.026794,11.982708,26.794150,0.007446,0.016649,39.313348,87.907318
6650,54312937,54312937,(2022 SW55),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54312937,25.23,False,"[{'close_approach_date': '2022-09-24', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54312937?a...,0.023909,0.053461,23.908646,53.461357,0.014856,0.033219,78.440441,175.398159


Extracting nested close approach data

In [8]:
json_df['close_approach_data'][0]

[{'close_approach_date': '2023-10-01',
  'close_approach_date_full': '2023-Oct-01 17:21',
  'epoch_date_close_approach': 1696180860000,
  'relative_velocity': {'kilometers_per_second': '18.0203381913',
   'kilometers_per_hour': '64873.2174887045',
   'miles_per_hour': '40309.6945001342'},
  'miss_distance': {'astronomical': '0.4544143686',
   'lunar': '176.7671893854',
   'kilometers': '67979421.639954882',
   'miles': '42240453.8911940916'},
  'orbiting_body': 'Earth'}]

Converting list of dict to dict

In [9]:
for n in json_df.index.values:
        json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}

json_df

C:\Users\Jack\AppData\Local\Temp\ipykernel_292\523775224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}


,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max
0,2137084,2137084,137084 (1998 XS16),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2137084,16.49,False,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/2137084?ap...,1.338305,2.992540,1338.304618,2992.540101,0.831584,1.859478,4390.763323,9818.045264
1,2252399,2252399,252399 (2001 TX44),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2252399,19.45,True,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/2252399?ap...,0.342417,0.765667,342.416731,765.667087,0.212768,0.475763,1123.414507,2512.031205
2,3342645,3342645,(2006 SG7),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3342645,22.82,False,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3342645?ap...,0.072536,0.162196,72.536229,162.195940,0.045072,0.100784,237.979763,532.138928
3,3451020,3451020,(2009 EG3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3451020,21.31,False,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3451020?ap...,0.145397,0.325117,145.396843,325.117225,0.090345,0.202018,477.023778,1066.657595
4,3606252,3606252,(2012 OO),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3606252,20.65,True,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3606252?ap...,0.197040,0.440595,197.040262,440.595420,0.122435,0.273773,646.457574,1445.523079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6647,54305672,54305672,(2022 SK5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54305672,23.69,False,"{'close_approach_date': '2022-09-24', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54305672?a...,0.048591,0.108653,48.590904,108.652564,0.030193,0.067514,159.418981,356.471678
6648,54308110,54308110,(2022 SF19),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54308110,29.31,False,"{'close_approach_date': '2022-09-24', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54308110?a...,0.003652,0.008167,3.652204,8.166575,0.002269,0.005074,11.982296,26.793227
6649,54311855,54311855,(2022 SL48),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54311855,26.73,False,"{'close_approach_date': '2022-09-24', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54311855?a...,0.011983,0.026794,11.982708,26.794150,0.007446,0.016649,39.313348,87.907318
6650,54312937,54312937,(2022 SW55),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54312937,25.23,False,"{'close_approach_date': '2022-09-24', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54312937?a...,0.023909,0.053461,23.908646,53.461357,0.014856,0.033219,78.440441,175.398159


In [10]:
cad_df = pd.json_normalize(json_df['close_approach_data'])

cad_df

,close_approach_date,close_approach_date_full,epoch_date_close_approach,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,relative_velocity.miles_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,miss_distance.miles
0,2023-10-01,2023-Oct-01 17:21,1696180860000,Earth,18.0203381913,64873.2174887045,40309.6945001342,0.4544143686,176.7671893854,67979421.639954882,42240453.8911940916
1,2023-10-01,2023-Oct-01 23:55,1696204500000,Earth,13.1737562484,47425.5224941968,29468.3753520191,0.4153101202,161.5556367578,62129509.371363974,38605486.9631478812
2,2023-10-01,2023-Oct-01 10:30,1696156200000,Earth,11.207421835,40346.7186060047,25069.8819027144,0.3264809883,127.0011044487,48840860.445174921,30348303.4111030698
3,2023-10-01,2023-Oct-01 06:34,1696142040000,Earth,19.7494474735,71098.0109047015,44177.5390535352,0.1343616471,52.2666807219,20100216.215851677,12489695.2016680626
4,2023-10-01,2023-Oct-01 23:20,1696202400000,Earth,9.4421483153,33991.7339350502,21121.1415664761,0.1685102646,65.5504929294,25208776.657296402,15664007.4651662676
...,...,...,...,...,...,...,...,...,...,...,...
6647,2022-09-24,2022-Sep-24 20:31,1664051460000,Earth,10.2392083239,36861.1499659532,22904.085099678,0.1892374522,73.6133689058,28309519.773346814,17590719.8946374732
6648,2022-09-24,2022-Sep-24 23:56,1664063760000,Earth,12.6899477354,45683.8118472928,28386.1440892248,0.0011272196,0.4384884244,168629.651182252,104781.6064569976
6649,2022-09-24,2022-Sep-24 00:42,1663980120000,Earth,9.1930363093,33094.930713458,20563.9029202595,0.0055532838,2.1602273982,830759.427985506,516209.9715757428
6650,2022-09-24,2022-Sep-24 10:28,1664015280000,Earth,11.7894933266,42442.1759756534,26371.9176222053,0.0348942843,13.5738765927,5220110.606454441,3243626.3218996458


In [11]:
df = pd.concat([json_df, cad_df, df], axis=1)

df

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,...,epoch_date_close_approach,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,relative_velocity.miles_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,miss_distance.miles,data
0,2137084,2137084,137084 (1998 XS16),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2137084,16.49,False,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/2137084?ap...,1.338305,...,1696180860000,Earth,18.0203381913,64873.2174887045,40309.6945001342,0.4544143686,176.7671893854,67979421.639954882,42240453.8911940916,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,2252399,2252399,252399 (2001 TX44),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2252399,19.45,True,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/2252399?ap...,0.342417,...,1696204500000,Earth,13.1737562484,47425.5224941968,29468.3753520191,0.4153101202,161.5556367578,62129509.371363974,38605486.9631478812,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3342645,3342645,(2006 SG7),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3342645,22.82,False,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3342645?ap...,0.072536,...,1696156200000,Earth,11.207421835,40346.7186060047,25069.8819027144,0.3264809883,127.0011044487,48840860.445174921,30348303.4111030698,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3451020,3451020,(2009 EG3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3451020,21.31,False,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3451020?ap...,0.145397,...,1696142040000,Earth,19.7494474735,71098.0109047015,44177.5390535352,0.1343616471,52.2666807219,20100216.215851677,12489695.2016680626,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,3606252,3606252,(2012 OO),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3606252,20.65,True,"{'close_approach_date': '2023-10-01', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3606252?ap...,0.197040,...,1696202400000,Earth,9.4421483153,33991.7339350502,21121.1415664761,0.1685102646,65.5504929294,25208776.657296402,15664007.4651662676,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6647,54305672,54305672,(2022 SK5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54305672,23.69,False,"{'close_approach_date': '2022-09-24', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54305672?a...,0.048591,...,1664051460000,Earth,10.2392083239,36861.1499659532,22904.085099678,0.1892374522,73.6133689058,28309519.773346814,17590719.8946374732,{'links': {'self': 'http://api.nasa.gov/neo/re...
6648,54308110,54308110,(2022 SF19),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54308110,29.31,False,"{'close_approach_date': '2022-09-24', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54308110?a...,0.003652,...,1664063760000,Earth,12.6899477354,45683.8118472928,28386.1440892248,0.0011272196,0.4384884244,168629.651182252,104781.6064569976,{'links': {'self': 'http://api.nasa.gov/neo/re...
6649,54311855,54311855,(2022 SL48),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54311855,26.73,False,"{'close_approach_date': '2022-09-24', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54311855?a...,0.011983,...,1663980120000,Earth,9.1930363093,33094.930713458,20563.9029202595,0.0055532838,2.1602273982,830759.427985506,516209.9715757428,{'links': {'self': 'http://api.nasa.gov/neo/re...
6650,54312937,54312937,(2022 SW55),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54312937,25.23,False,"{'close_approach_date': '2022-09-24', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54312937?a...,0.023909,...,1664015280000,Earth,11.7894933266,42442.1759756534,26371.9176222053,0.0348942843,13.573876592

Cleaning up df

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6652 entries, 0 to 6651
Data columns (total 29 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   id                                                    6652 non-null   object 
 1   neo_reference_id                                      6652 non-null   object 
 2   name                                                  6652 non-null   object 
 3   nasa_jpl_url                                          6652 non-null   object 
 4   absolute_magnitude_h                                  6652 non-null   float64
 5   is_potentially_hazardous_asteroid                     6652 non-null   bool   
 6   close_approach_data                                   6652 non-null   object 
 7   is_sentry_object                                      6652 non-null   bool   
 8   links.self                                            6652

In [13]:
df = df.drop(['neo_reference_id', 'close_approach_data', 'links.self',
        'estimated_diameter.kilometers.estimated_diameter_min',
        'estimated_diameter.kilometers.estimated_diameter_max',
        'estimated_diameter.miles.estimated_diameter_min',
        'estimated_diameter.miles.estimated_diameter_max',
        'estimated_diameter.feet.estimated_diameter_min',
        'estimated_diameter.feet.estimated_diameter_max',
        'close_approach_date', 'epoch_date_close_approach',
        'relative_velocity.miles_per_hour',
        'miss_distance.miles'], axis=1
        )

df

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,close_approach_date_full,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,data
0,2137084,137084 (1998 XS16),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2137084,16.49,False,False,1338.304618,2992.540101,2023-Oct-01 17:21,Earth,18.0203381913,64873.2174887045,0.4544143686,176.7671893854,67979421.639954882,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,2252399,252399 (2001 TX44),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2252399,19.45,True,False,342.416731,765.667087,2023-Oct-01 23:55,Earth,13.1737562484,47425.5224941968,0.4153101202,161.5556367578,62129509.371363974,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3342645,(2006 SG7),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3342645,22.82,False,False,72.536229,162.195940,2023-Oct-01 10:30,Earth,11.207421835,40346.7186060047,0.3264809883,127.0011044487,48840860.445174921,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3451020,(2009 EG3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3451020,21.31,False,False,145.396843,325.117225,2023-Oct-01 06:34,Earth,19.7494474735,71098.0109047015,0.1343616471,52.2666807219,20100216.215851677,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,3606252,(2012 OO),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3606252,20.65,True,False,197.040262,440.595420,2023-Oct-01 23:20,Earth,9.4421483153,33991.7339350502,0.1685102646,65.5504929294,25208776.657296402,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6647,54305672,(2022 SK5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54305672,23.69,False,False,48.590904,108.652564,2022-Sep-24 20:31,Earth,10.2392083239,36861.1499659532,0.1892374522,73.6133689058,28309519.773346814,{'links': {'self': 'http://api.nasa.gov/neo/re...
6648,54308110,(2022 SF19),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54308110,29.31,False,False,3.652204,8.166575,2022-Sep-24 23:56,Earth,12.6899477354,45683.8118472928,0.0011272196,0.4384884244,168629.651182252,{'links': {'self': 'http://api.nasa.gov/neo/re...
6649,54311855,(2022 SL48),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54311855,26.73,False,False,11.982708,26.794150,2022-Sep-24 00:42,Earth,9.1930363093,33094.930713458,0.0055532838,2.1602273982,830759.427985506,{'links': {'self': 'http://api.nasa.gov/neo/re...
6650,54312937,(2022 SW55),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54312937,25.23,False,False,23.908646,53.461357,2022-Sep-24 10:28,Earth,11.7894933266,42442.1759756534,0.0348942843,13.5738765927,5220110.606454441,{'links': {'self': 'http://api.nasa.gov/neo/re...


In [14]:
df = df.rename(columns={
    'estimated_diameter.meters.estimated_diameter_min': 'estimated_diameter.meters_min',
    'estimated_diameter.meters.estimated_diameter_max': 'estimated_diameter.meters_max'
    })
df

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter.meters_min,estimated_diameter.meters_max,close_approach_date_full,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,data
0,2137084,137084 (1998 XS16),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2137084,16.49,False,False,1338.304618,2992.540101,2023-Oct-01 17:21,Earth,18.0203381913,64873.2174887045,0.4544143686,176.7671893854,67979421.639954882,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,2252399,252399 (2001 TX44),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2252399,19.45,True,False,342.416731,765.667087,2023-Oct-01 23:55,Earth,13.1737562484,47425.5224941968,0.4153101202,161.5556367578,62129509.371363974,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3342645,(2006 SG7),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3342645,22.82,False,False,72.536229,162.195940,2023-Oct-01 10:30,Earth,11.207421835,40346.7186060047,0.3264809883,127.0011044487,48840860.445174921,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3451020,(2009 EG3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3451020,21.31,False,False,145.396843,325.117225,2023-Oct-01 06:34,Earth,19.7494474735,71098.0109047015,0.1343616471,52.2666807219,20100216.215851677,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,3606252,(2012 OO),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3606252,20.65,True,False,197.040262,440.595420,2023-Oct-01 23:20,Earth,9.4421483153,33991.7339350502,0.1685102646,65.5504929294,25208776.657296402,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6647,54305672,(2022 SK5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54305672,23.69,False,False,48.590904,108.652564,2022-Sep-24 20:31,Earth,10.2392083239,36861.1499659532,0.1892374522,73.6133689058,28309519.773346814,{'links': {'self': 'http://api.nasa.gov/neo/re...
6648,54308110,(2022 SF19),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54308110,29.31,False,False,3.652204,8.166575,2022-Sep-24 23:56,Earth,12.6899477354,45683.8118472928,0.0011272196,0.4384884244,168629.651182252,{'links': {'self': 'http://api.nasa.gov/neo/re...
6649,54311855,(2022 SL48),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54311855,26.73,False,False,11.982708,26.794150,2022-Sep-24 00:42,Earth,9.1930363093,33094.930713458,0.0055532838,2.1602273982,830759.427985506,{'links': {'self': 'http://api.nasa.gov/neo/re...
6650,54312937,(2022 SW55),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54312937,25.23,False,False,23.908646,53.461357,2022-Sep-24 10:28,Earth,11.7894933266,42442.1759756534,0.0348942843,13.5738765927,5220110.606454441,{'links': {'self': 'http://api.nasa.gov/neo/re...


Adding date awareness to allow for updating data without duplicates and loading previous data

In [15]:
datetime.date.today() - datetime.timedelta(days=365)

datetime.date(2022, 10, 1)

In [16]:
try:
    with open('last_executed.txt') as f:
        last_execution_date = datetime.datetime.strptime(f.read(), '%Y-%m-%d').date()
except FileNotFoundError:
        last_execution_date = (datetime.date.today() - datetime.timedelta(days=365))

last_execution_date

datetime.date(2023, 10, 1)

In [17]:
delta = (datetime.date.today() - last_execution_date).days
delta

0

In [18]:
# while delta % 7 != 0:
#     variable = (datetime.date.today() - datetime.timedelta(delta%7))
#     date = variable.strftime('%Y-%m-%d')

#     link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={date}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
    
#     delta = delta - delta % 7
#     print(link)
# while delta // 7 != 0:
#     variable = variable - datetime.timedelta(days=7)
#     date = variable.strftime('%Y-%m-%d')

#     link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={date}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
    
#     delta = delta - 7
#     print(link)

In [52]:
delta = 7

for x in range (delta%7, delta, 7):
    print(x)

0


In [73]:
delta = 8
def get_links():
    if delta == 0:
        return []

    links = []
    
    if delta % 7 != 0:
        end_date = datetime.date.today()
        start_date = (end_date - datetime.timedelta(days=delta%7)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

    if delta == 7:
        end_date = datetime.date.today()
        start_date = (end_date - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)
    else:
        for x in range(delta%7, delta, 7):
            end_date = (datetime.date.today() - datetime.timedelta(days=x+1))
            start_date = (end_date - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
            end_date = end_date.strftime('%Y-%m-%d')
            link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
            links.append(link)   

    return links

get_links()            

['https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-30&end_date=2023-10-01&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-22&end_date=2023-09-29&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [21]:
def load_data():
    
    for link in links:
    
        response = requests.get(link)
        data = response.json()
        neo_data = data['near_earth_objects']

        cursor = conn.cursor()

        cursor.executescript('''

             CREATE TABLE IF NOT EXISTS neo (
                            id text,
                            data json
            )
        ''')

        for date, neo_list in neo_data.items():
            for neo in neo_list:
                cursor.execute('INSERT INTO neo VALUES (?, ?)',
                            (neo['id'], json.dumps(neo)))
        conn.commit()

        with open('last_executed.txt', 'w') as f:
            f.write(str(datetime.date.today()))

In [22]:
from main import transform

load_data(), transform()

c:\Users\Jack\Desktop\Coding\NASA\main.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}


            id               name  \
0      2164207  164207 (2004 GU9)   
1      2504800  504800 (2010 CO1)   
2      3401428         (2008 CL1)   
3      3748455        (2016 FA13)   
4      3773752         (2017 GK4)   
...        ...                ...   
7711   3843536         (2019 PM2)   
7712   3989291        (2020 BD13)   
7713  54051300         (2020 QG3)   
7714  54115824         (2021 CN5)   
7715  54336908         (2022 YL5)   

                                        nasa_jpl_url  absolute_magnitude_h  \
0      http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2164207                 21.19   
1      http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2504800                 21.48   
2      http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3401428                 19.49   
3      http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3748455                 22.50   
4      http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3773752                 21.79   
...                                              ...                   ...   
7711   ht

(None, None)